# Huấn luyện và đánh giá các mô hình phân loại DR trên tập aptos2019
Notebook này thực hiện:
- Chia tập dữ liệu thành train, validation, test.
- Huấn luyện EfficientNet_B3 (50 epochs), ResNet18, ResNet34, ResNet50 (10 epochs mỗi mô hình).
- Đánh giá mô hình trên tập test và so sánh kết quả.
- Minh họa confusion matrix, loss/accuracy curve.
- Đánh giá các chỉ số: Accuracy, Precision, Recall, F1-Score, MAE, MSE.
- So sánh với baseline.

In [10]:
# 1. Import Required Libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# 2. Load and Preprocess Dataset
DATA_DIR = 'aptos2019/train_images'
CSV_PATH = 'aptos2019/train.csv'
IMG_SIZE = 300
NUM_CLASSES = 5
df = pd.read_csv(CSV_PATH)
class DRDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'] + '.png')
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image: {img_path}, error: {e}")
            image = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (0, 0, 0))
        label = int(row['diagnosis'])
        if self.transform:
            image = self.transform(image)
        return image, label
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [12]:
# 3. Split Dataset into Train, Validation, and Test Sets
train_df, test_df = train_test_split(df, test_size=0.15, stratify=df['diagnosis'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df['diagnosis'], random_state=42)
print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

Train: 2645, Val: 467, Test: 550


In [13]:
# 4. Define Data Loaders
BATCH_SIZE = 32
train_dataset = DRDataset(train_df, DATA_DIR, transform)
val_dataset = DRDataset(val_df, DATA_DIR, transform)
test_dataset = DRDataset(test_df, DATA_DIR, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [14]:
# 5. Model Definitions
def get_efficientnet_b3(num_classes):
    model = models.efficientnet_b3(weights=None)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model
def get_resnet(model_name, num_classes):
    model = getattr(models, model_name)(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

In [15]:
# 6. Training Function
def train_model(model, train_loader, val_loader, epochs, lr, device, model_name):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_val_loss = float('inf')
    train_losses, val_losses, val_accuracies = [], [], []
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in tqdm(train_loader, desc=f'Train Epoch {epoch+1}'):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(1)
                correct += (preds == labels).sum().item()
        val_loss /= len(val_loader.dataset)
        val_acc = correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'{model_name}_best.pth')
            print('Saved best model!')
    return train_losses, val_losses, val_accuracies

In [16]:
# 7. Train EfficientNet_B3 (50 epochs)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
efficientnet_b3 = get_efficientnet_b3(NUM_CLASSES)
print('--- Training EfficientNet_B3 ---')
train_losses_b3, val_losses_b3, val_acc_b3 = train_model(efficientnet_b3, train_loader, val_loader, epochs=50, lr=1e-4, device=device, model_name='EfficientNet_B3')

--- Training EfficientNet_B3 ---


Train Epoch 1: 100%|██████████| 83/83 [09:17<00:00,  6.71s/it]


Epoch 1: Train Loss=1.2138, Val Loss=1.0967, Val Acc=0.7173
Saved best model!


Train Epoch 2: 100%|██████████| 83/83 [09:12<00:00,  6.66s/it]


Epoch 2: Train Loss=0.8835, Val Loss=0.9186, Val Acc=0.6467
Saved best model!


...

Epoch 5: Train Loss=0.7237, Val Loss=0.8617, Val Acc=0.7131


Train Epoch 6: 100%|██████████| 83/83 [09:39<00:00,  6.99s/it]


Epoch 6: Train Loss=0.6932, Val Loss=0.8037, Val Acc=0.7173


Train Epoch 7:  52%|█████▏    | 43/83 [05:01<04:40,  7.01s/it]


KeyboardInterrupt: 

In [ ]:
# 8. Train ResNet18 (10 epochs)
resnet18 = get_resnet('resnet18', NUM_CLASSES)
print('--- Training ResNet18 ---')
train_losses_r18, val_losses_r18, val_acc_r18 = train_model(resnet18, train_loader, val_loader, epochs=10, lr=1e-4, device=device, model_name='ResNet18')

In [ ]:
# 9. Train ResNet34 (10 epochs)
resnet34 = get_resnet('resnet34', NUM_CLASSES)
print('--- Training ResNet34 ---')
train_losses_r34, val_losses_r34, val_acc_r34 = train_model(resnet34, train_loader, val_loader, epochs=10, lr=1e-4, device=device, model_name='ResNet34')

In [ ]:
# 10. Train ResNet50 (10 epochs)
resnet50 = get_resnet('resnet50', NUM_CLASSES)
print('--- Training ResNet50 ---')
train_losses_r50, val_losses_r50, val_acc_r50 = train_model(resnet50, train_loader, val_loader, epochs=10, lr=1e-4, device=device, model_name='ResNet50')

In [ ]:
# 11. Đánh giá mô hình trên test set
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error
def evaluate_model(model, test_loader, device):
    model = model.to(device)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_preds)
results = {}
for name, model in zip(['EfficientNet_B3','ResNet18','ResNet34','ResNet50'], [efficientnet_b3, resnet18, resnet34, resnet50]):
    model.load_state_dict(torch.load(f'{name}_best.pth'))
    y_true, y_pred = evaluate_model(model, test_loader, device)
    acc = precision_score(y_true, y_pred, average='macro')
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    results[name] = {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1-Score': f1, 'MAE': mae, 'MSE': mse}
    print(f'--- {name} ---')
    print(classification_report(y_true, y_pred))
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
# 12. Bảng so sánh kết quả các mô hình
results_df = pd.DataFrame(results).T
display(results_df)

In [ ]:
# 13. Baseline: dự đoán lớp phổ biến nhất
from collections import Counter
most_common = Counter(y_true).most_common(1)[0][0]
baseline_preds = [most_common] * len(y_true)
baseline_acc = precision_score(y_true, baseline_preds, average='macro')
baseline_prec = precision_score(y_true, baseline_preds, average='macro')
baseline_rec = recall_score(y_true, baseline_preds, average='macro')
baseline_f1 = f1_score(y_true, baseline_preds, average='macro')
baseline_mae = mean_absolute_error(y_true, baseline_preds)
baseline_mse = mean_squared_error(y_true, baseline_preds)
results_df.loc['Baseline'] = [baseline_acc, baseline_prec, baseline_rec, baseline_f1, baseline_mae, baseline_mse]
display(results_df)

In [ ]:
# 14. Vẽ loss/accuracy curve cho từng mô hình
plt.figure(figsize=(12,6))
plt.plot(train_losses_b3, label='EffNetB3 Train Loss')
plt.plot(val_losses_b3, label='EffNetB3 Val Loss')
plt.plot(train_losses_r18, label='ResNet18 Train Loss')
plt.plot(val_losses_r18, label='ResNet18 Val Loss')
plt.plot(train_losses_r34, label='ResNet34 Train Loss')
plt.plot(val_losses_r34, label='ResNet34 Val Loss')
plt.plot(train_losses_r50, label='ResNet50 Train Loss')
plt.plot(val_losses_r50, label='ResNet50 Val Loss')
plt.legend()
plt.title('Loss Curves')
plt.show()
plt.figure(figsize=(12,6))
plt.plot(val_acc_b3, label='EffNetB3 Val Acc')
plt.plot(val_acc_r18, label='ResNet18 Val Acc')
plt.plot(val_acc_r34, label='ResNet34 Val Acc')
plt.plot(val_acc_r50, label='ResNet50 Val Acc')
plt.legend()
plt.title('Validation Accuracy Curves')
plt.show()

### Đánh giá tổng quan
- So sánh các mô hình dựa trên bảng kết quả.
- Phân tích confusion matrix để nhận biết các lớp dễ nhầm lẫn.
- Đánh giá các chỉ số: Accuracy, Precision, Recall, F1-Score, MAE, MSE.
- So sánh với baseline để kiểm tra hiệu quả mô hình.